In [1]:
### Ligand file creation - how exactly do we generate the SDF files for novel lignands?

import os
import subprocess
from SBLMDCOVDOCK.SBLSettings import GROMACS_Settings
import requests
import pandas as pd
settings = GROMACS_Settings()

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"]})



In [2]:
# use: https://ambermd.org/tutorials/basic/tutorial5/index.php
# only partially works with GAFF - doesnt generate charges.

resname = "FYG"

# download from PDB
# 
link = f"https://files.rcsb.org/ligands/download/{resname}.cif"

# download the file to FYG_dir

FYG_dir = os.path.join(settings.structures_input, resname)

# Check if the directory exists, if not, create it
# if not os.path.exists(FYG_dir):
os.makedirs(FYG_dir,exist_ok=True)

# Download the CIF file and save it to the target directory
cif_file_path = os.path.join(FYG_dir, f"{resname}.cif")
response = requests.get(link)

if response.status_code == 200:
    with open(cif_file_path, "wb") as f:
        f.write(response.content)
    print(f"CIF file downloaded and saved to {cif_file_path}")
else:
    print("Failed to download the CIF file.")

CIF file downloaded and saved to start_structures/APO/FYG/FYG.cif


In [3]:
# Change O


In [4]:


print(FYG_dir)

cif_name = resname + ".cif"

antechamber_command = [
    'antechamber',
    '-i', cif_name,
    '-fi', 'ccif',
    "-bk", resname, # component block for ccif
    '-o', cif_name.replace('.cif', '.mol2'),
    '-fo', 'mol2',
    '-c', 'bcc', # charge method
    '-at', 'amber', # ff atom types
    '-s', '2', # verbose
    '-nc', '0'

]

print(" ".join(antechamber_command))

subprocess.run(antechamber_command, check=True, cwd=FYG_dir)

start_structures/APO/FYG
antechamber -i FYG.cif -fi ccif -bk FYG -o FYG.mol2 -fo mol2 -c bcc -at amber -s 2 -nc 0

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

Info: Determining atomic numbers from atomic symbols which are case sensitive.
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Running: /Users/alexi/miniconda/envs/RIN_test/bin/bondtype -j full -i ANTECHAMBER_BOND_TYPE.AC0 -o ANTECHAMBER_BOND_TYPE.AC -f ac

Running: /Users/alexi/miniconda/envs/RIN_test/bin/atomtype -i ANTECHAMBER_AC.AC0 -o ANTECHAMBER_AC.

CompletedProcess(args=['antechamber', '-i', 'FYG.cif', '-fi', 'ccif', '-bk', 'FYG', '-o', 'FYG.mol2', '-fo', 'mol2', '-c', 'bcc', '-at', 'amber', '-s', '2', '-nc', '0'], returncode=0)

In [8]:
# edit mol2 to remove protonprint(FYG_dir)
parkchk_command = ["parmchk2",
                    "-i", resname + "_noHO" + ".mol2",
                    "-f", "mol2",
                    "-o", resname + "_noHO" + '.frcmod',
]

print(" ".join(parkchk_command))

subprocess.run(parkchk_command, check=True, cwd=FYG_dir)
print(FYG_dir)

parmchk2 -i FYG_noHO.mol2 -f mol2 -o FYG_noHO.frcmod
start_structures/APO/FYG


In [7]:
print(FYG_dir)
parkchk_command = ["parmchk2",
                    "-i", cif_name.replace('.cif', '.mol2'),
                    "-f", "mol2",
                    "-o", cif_name.replace('.cif', '.frcmod'),
]

print(" ".join(parkchk_command))

subprocess.run(parkchk_command, check=True, cwd=FYG_dir)


start_structures/APO/FYG
parmchk2 -i FYG.mol2 -f mol2 -o FYG.frcmod


CompletedProcess(args=['parmchk2', '-i', 'FYG.mol2', '-f', 'mol2', '-o', 'FYG.frcmod'], returncode=0)

In [48]:
ff_name = "ff14SB"
print(FYG_dir)
tleap_command = [
    'tleap',
    "-f", " oldff/leaprc."+ff_name]
print(" ".join(tleap_command))

start_structures/APO/FYG
tleap -f  oldff/leaprc.ff14SB


In [53]:
leap_commands = [
    "source oldff/leaprc."+ff_name,
    "source leaprc.gaff",
    f"{resname} = loadmol2 {resname}.mol2",
    f"loadamberparams {resname}.frcmod",
    f"check {resname}",
    f"saveoff {resname} {resname}.lib",
    f"saveamberparm {resname} {resname}.prmtop {resname}.inpcrd",
    f"savepdb {resname} {resname}.pdb",
    "quit"
]

leap_file = "\n".join(leap_commands)

with open(os.path.join(FYG_dir, "leap.in"), "w+") as f:
    f.write(leap_file)
tleap_command = ["tleap", "-f", "leap.in"]

subprocess.run(tleap_command,cwd=FYG_dir,check=True)

for command in leap_commands:
    print(command)

-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/prep to search path.
-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/lib to search path.
-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm to search path.
-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/cmd to search path.
-f: Source leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./leap.in
----- Source: /Users/alexi/miniconda/envs/RIN_test/dat/leap/cmd/oldff/leaprc.ff14SB
----- Source of /Users/alexi/miniconda/envs/RIN_test/dat/leap/cmd/oldff/leaprc.ff14SB done
Log file: ./leap.log
Loading parameters: /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /Users/alexi/miniconda/envs/RIN_test/da

In [57]:
### Add Ligand to structure
for pdb in structures.PDBID:
    print(pdb)
    pdb_name = "APO_" + pdb + "_chimeraFH" + "_CYX"+ ".pdb"
    pdb_path = os.path.join(settings.structures_input, pdb_name)
    print(pdb_path)

    ligand_name = resname + ".pdb"
    ligand_path = os.path.join(FYG_dir, ligand_name)
    
    FYG_name = f"{resname}_" + pdb + "_chimeraFH" + "_CYX"+ ".pdb"
    FYG_path = os.path.join(settings.structures_input, FYG_name)
    print(FYG_path)

    # load the protein structure
    with open(pdb_path, "r") as f:
        pdb_file = f.readlines()

    # load the ligand structure
    with open(ligand_path, "r") as f:
        ligand_file = f.readlines()

    # add the ligand to the protein structure
    with open(FYG_path, "w+") as f:
        f.writelines(pdb_file)
        f.writelines(ligand_file)
        
    

    

6T3D
start_structures/APO/APO_6T3D_chimeraFH_CYX.pdb
start_structures/APO/FYG_6T3D_chimeraFH_CYX.pdb
5UL8
start_structures/APO/APO_5UL8_chimeraFH_CYX.pdb
start_structures/APO/FYG_5UL8_chimeraFH_CYX.pdb
1K55
start_structures/APO/APO_1K55_chimeraFH_CYX.pdb
start_structures/APO/FYG_1K55_chimeraFH_CYX.pdb


In [ ]:
for pdb in structures.PDBID:
    print(pdb)
    pdb_name = "APO_" + "_chimeraFH" + "_CYX"+ ".pdb"
    print(pdb_name)
    output_pdb = resname + "_" + pdb + ".pdb"

    leap_file = f"""
    source leaprc.gaff
    loadamberparams {resname}.frcmod
    loadoff {resname}.lib
    complex = loadpdb {pdb_name}
    saveamberparm complex {output_pdb.replace(".pdb", ".prmtop")} {output_pdb.replace(".pdb", ".inpcrd")} {output_pdb.replace(".pdb", ".rst7")}
    savepdb complex {output_pdb}
    quit
    """

    with open(os.path.join(FYG_dir, "leap.in"), "w+") as f:
        f.write(leap_file)
    
    leap_command = [
        'tleap',
        "-f", "leap.in"]
    print(" ".join(leap_command))
    subprocess.run(leap_command, check=True, cwd=FYG_dir)
    



In [ ]:
# find
!open $AMBERHOME

In [ ]:
# adapt parmchk2 -i cro.prepin -f prepi -o frcmod.cro -a Y -p $AMBERHOME/dat/leap/parm/parm10.dat

ff_dat = "parm19.dat"

parmchk_command = [
    "parmchk2",
    "-i", resname + ".prepin",
    "-f", "prepi",
    "-o", "frcmod." + resname,
    "-a", "Y",
    "-p", os.environ["AMBERHOME"]+"/dat/leap/parm/"+ ff_dat
]

print(" ".join(parmchk_command))
subprocess.run(parmchk_command, check=True, cwd=KCX_dir)

In [ ]:
# strip out ATTN lines

# ! grep -v "ATTN" {"frcmod."+resname} > {"frcmod1."+resname} # Strip out ATTN lines

grep_command = [
    "grep", "-v", "ATTN",
    os.path.join(KCX_dir, "frcmod."+resname),
    ">", os.path.join(KCX_dir, "frcmod1."+resname)
]

# Run grep command
print(" ".join(grep_command))
! {" ".join(grep_command)}
# subprocess.run(grep_command, check=True)

In [ ]:
# GAFF parameters for ligands
# adapt parmchk2 -i cro.prepin -f prepi -o frcmod.cro 


parmchk_command = [
    "parmchk2",
    "-i", resname + ".prepin",
    "-f", "prepi",
    "-o", "frcmod2." + resname,
    # "-a", "Y",
    # "-p", os.environ["AMBERHOME"]+"/dat/leap/parm/"+ ff_dat
]

print(" ".join(parmchk_command))
subprocess.run(parmchk_command, check=True, cwd=KCX_dir)

In [ ]:

leap_file = f"""source leaprc.protein.ff19SB
set default PBRadii mbondi3
loadAmberPrep {resname}.prepin
loadAmberParams frcmod2.{resname}
loadAmberParams frcmod1.{resname}
x = loadPDB gfp.pdb
saveAmberParm x gfp.parm7 gfp.rst7
quit
"""